In [3]:
pip install groq

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install streamlit

PermissionError: [WinError 5] Access is denied

In [2]:
pip install pypyodbc


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pypyodbc: filename=pypyodbc-1.3.6-py3-none-any.whl size=22871 sha256=585ca1f62a18dd81fa9268f10ff58749bd7f7953def2feb77d038b22b444c09b
  Stored in directory: c:\users\saiga\appdata\local\pip\cache\wheels\5d\fa\2f\becd808255e1b61b755eb3a3aa256229a7d7e1fc89e2aef2bc
Successfully built pypyodbc


In [46]:
#connecting with SSMS
import pypyodbc as podbc
import os
import pandas as pd

conn2 = podbc.connect ("Driver={ODBC Driver 17 for SQL Server};"
                       "Server=localhost;"
                       "Database=business_data;"
                       "Trusted_Connection=yes;")

SQL_Query = pd.read_sql_query("SELECT * FROM dbo.product_info", conn2)

SQL_Query.head()

C:\Users\saiga\AppData\Local\Temp\ipykernel_19780\3135425017.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SQL_Query = pd.read_sql_query("SELECT * FROM dbo.product_info", conn2)


,product_id,name,features,price
0,1,Nice Max50,"threat, among, attorney, very, safe, Culture t...",977.47
1,2,Direction Ultra39,"food, yes, charge, body, account, Stuff howeve...",749.46
2,3,Figure Ultra5,"grow, side, development, when, determine, Able...",948.99
3,4,Fish X30,"space, else, interest, style, little, Happen f...",165.09
4,5,Fill Ultra10,"successful, choice, special, data, thus, Speci...",1254.44


In [48]:
import os
from groq import Groq

client = Groq(api_key='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')

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content":"you are a helpful assistant."
        },
        {
            "role":"user",
            "content":"Explain the importance of fast language models",
        }
    ],
    model="llama-3.1-8b-instant",
)

print(chat_completion.choices[0].message.content)

Fast language models are a type of artificial intelligence (AI) model that is designed to process and generate human-like text or speech quickly and efficiently. The importance of fast language models can be seen in various applications across different industries.

**Key Importance Factors:**

1. **Efficient NLP Processing**: Fast language models enable the rapid analysis and understanding of vast amounts of text data, which is crucial in natural language processing (NLP) applications such as text classification, sentiment analysis, and machine translation.
2. **Real-time Interaction**: Fast language models enable real-time text or speech interaction, making them ideal for applications such as chatbots, voice assistants, and virtual customer support agents.
3. **Speed and Efficiency**: Fast language models are designed to process large amounts of text quickly, making them suitable for applications that require rapid text analysis and generation, such as real-time feedback systems and 

In [49]:
import os
from groq import Groq

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content":"you are a helpful assistant."
        },
        {
            "role":"user",
            "content":"Explain the importance of large language models",
        }
    ],
    model="llama-3.1-8b-instant",
)

print(chat_completion.choices[0].message.content)

Large language models (LLMs) have become increasingly important in recent years due to their ability to process and generate human-like language. Some of the key importance of LLMs include:

1. **Natural Language Processing (NLP)**: LLMs are a crucial component of NLP, which is the field of study that focuses on enabling computers to understand, interpret, and generate human language. LLMs are used in various NLP tasks such as sentiment analysis, named entity recognition, and text classification.
2. **Conversational AI**: LLMs are used to build conversational AI systems such as chatbots, voice assistants, and virtual customer service agents. These systems can understand and respond to user queries, enabling businesses to provide 24/7 customer support.
3. **Language Translation**: LLMs are also used for language translation, which is a critical task in today's globalized economy. They can translate text from one language to another, enabling businesses and individuals to communicate eff

In [73]:
#creating agents
import pypyodbc
from groq import Groq
import re
import pandas as pd

example_sql_query = """
SELECT * FROM [dbo].[product_info];
"""

connection = pypyodbc.connect ("Driver={ODBC Driver 17 for SQL Server};"
                       "Server=localhost;"
                       "Database=business_data;"
                       "Trusted_Connection=yes;")
cursor = connection.cursor()

TABLE_NAME = "product_info"
SCHEMA_NAME = "[dbo]"

def extract_table_metadata():
    metadata_query= f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{TABLE_NAME}'"
    cursor.execute(metadata_query)
    columns=cursor.fetchall()
    column_metadata={f"[{col[0]}]":col[1] for col in columns}
    print(f"Extracted metadata for table '{TABLE_NAME}':{column_metadata}")
    example_sql_query=f"SELECT{', '.join(column_metadata.keys())} FROM {TABLE_NAME} WHERE [CONDITION] LIMIT 10;"
    return column_metadata, example_sql_query

In [75]:
TABLE_NAME = "product_info"
SCHEMA_NAME = "[dbo]"
extract_table_metadata()

Extracted metadata for table 'product_info':{'[product_id]': 'int', '[name]': 'varchar', '[features]': 'text', '[price]': 'decimal'}


({'[product_id]': 'int',
  '[name]': 'varchar',
  '[features]': 'text',
  '[price]': 'decimal'},
 'SELECT[product_id], [name], [features], [price] FROM product_info WHERE [CONDITION] LIMIT 10;')

In [77]:
#extract only SQL query from the response
def extract_sql_from_response(response):
    queries=re.findall(r"SELECT.*?;", response, re.IGNORECASE | re.DOTALL)
    if queries:
        return queries[0].strip().rstrip(';')
    else:
        raise ValueError("No valid SQL query found.")

In [79]:
#function to generate SQL query
def generate_sql_query(question, column_metadata, table_name, schema_name):
    full_table_name=f"{schema_name}.[{table_name}]"
    prompt=f"""Generate only 1 SQL query based on this question:{question}.
    The table to use is '{full_table_name}'. The columns are: {', '.join(column_metadata.keys())}.
    Use only the columns needed to answer the question, not all of them.
    Limit the data as much as possible and follows T-SQL syntax.
    Make sure to use the correct column names as provided with square brackets.
    Column [price] is money, so make sure you convert that to money before aggregating."""
    model_name='llama-3.1-8b-instant'
    chat_completion=client.chat.completions.create(
        messages=[
            {"role":"system", "content":"You are a helpful assistant."},
            {"role":"user", "content": prompt}
        ],
        model=model_name,
    )
    print(chat_completion.choices[0].message.content.strip())
    response=chat_completion.choices[0].message.content.strip()
    return extract_sql_from_response(response)

In [110]:
table_name="product_info"
schema_name="[dbo]"
column_metadata, example_sql_query = extract_table_metadata()
question="What are the top 10 costly mobiles in our data and their details?"

Extracted metadata for table 'product_info':{'[product_id]': 'int', '[name]': 'varchar', '[features]': 'text', '[price]': 'decimal'}


In [112]:
generate_sql_query(question, column_metadata, table_name, schema_name)

```sql
SELECT TOP 10 [product_id], [name], [price]
FROM (
    SELECT [product_id], [name], CONVERT(MONEY, [price]) AS price,
    -- Rank products by price, highest first
    ROW_NUMBER() OVER (ORDER BY [price] DESC) AS row_num
    FROM [dbo].[product_info]
) AS subquery
WHERE row_num <= 10;
```

This query uses a subquery to assign a row number to each product in descending order by price. The main query then only selects the top 10 products based on this ranking. The `CONVERT(MONEY, [price])` is used to convert the price to money (since the price data type is money), which is required when using it in the `ROW_NUMBER` function.


'SELECT TOP 10 [product_id], [name], [price]\nFROM (\n    SELECT [product_id], [name], CONVERT(MONEY, [price]) AS price,\n    -- Rank products by price, highest first\n    ROW_NUMBER() OVER (ORDER BY [price] DESC) AS row_num\n    FROM [dbo].[product_info]\n) AS subquery\nWHERE row_num <= 10'

In [114]:
#fetching data from SQL
def fetch_answer_from_db(sql_query):
    cursor.execute(sql_query)
    columns=[column[0] for column in cursor.description]
    result=cursor.fetchall()
    df=pd.DataFrame(result,columns=columns)
    return df

In [116]:
sql_query=generate_sql_query(question, column_metadata, table_name, schema_name)
fetch_answer_from_db(sql_query)

```sql
SELECT TOP 10
    [product_id],
    [name],
    CONVERT(money, CAST([price] AS DECIMAL(10, 2)))
AS Price
FROM 
    [dbo].[product_info]
ORDER BY Price DESC;
```

This SQL query will return the top 10 most expensive mobiles, ordered by price in descending order. The `CONVERT(money, ...)` function is used to ensure that the price is rounded to two decimal places and displayed as a money format.


,product_id,name,price
0,187,I X96,1489.5400
1,124,Work Max26,1468.4100
2,89,In Pro81,1461.7100
3,107,Financial Pro37,1457.1300
4,189,Child Ultra59,1455.5400
5,91,Environmental Pro34,1443.8600
6,20,Account Pro30,1441.4700
7,74,Money X56,1430.2100
8,190,Assume Ultra49,1413.2000
9,150,Whole X40,1391.0300


In [118]:
#agent where answer the question based on the DataFrame
def answer_question_from_df(question,df):
    df_json=df.to_json(orient='records')
    prompt=f"""Based on the following data, answer this question:{question}.
Here is the data:
{df_json}
Please provide a concise and accurate response."""
    model_name='llama-3.1-8b-instant'
    chat_completion=client.chat.completions.create(
        messages=[
            {"role":"system", "content":"You are a helpful assistant."},
            {"role":"user","content":prompt}
        ],
        model=model_name,
    )
    response=chat_completion.choices[0].message.content.strip()
    return response

In [120]:
df=fetch_answer_from_db(sql_query)
question="What are the top 10 costly mobiles in our data and their details?"
answer_question_from_df(question,df)

'Based on the given data, the top 10 costly mobiles and their details are as follows:\n\n1. I X96\n   - Product ID: 187\n   - Name: I X96\n   - Price: $1489.54\n\n2. Work Max26\n   - Product ID: 124\n   - Name: Work Max26\n   - Price: $1468.41\n\n3. In Pro81\n   - Product ID: 89\n   - Name: In Pro81\n   - Price: $1461.71\n\n4. Financial Pro37\n   - Product ID: 107\n   - Name: Financial Pro37\n   - Price: $1457.13\n\n5. Child Ultra59\n   - Product ID: 189\n   - Name: Child Ultra59\n   - Price: $1455.54\n\n6. Environmental Pro34\n   - Product ID: 91\n   - Name: Environmental Pro34\n   - Price: $1443.86\n\n7. Account Pro30\n   - Product ID: 20\n   - Name: Account Pro30\n   - Price: $1441.47\n\n8. Money X56\n   - Product ID: 74\n   - Name: Money X56\n   - Price: $1430.21\n\n9. Assume Ultra49\n   - Product ID: 190\n   - Name: Assume Ultra49\n   - Price: $1413.2\n\n10. Whole X40\n   - Product ID: 150\n   - Name: Whole X40\n   - Price: $1391.03'

In [126]:
import pypyodbc as podbc
import os
import pandas as pd
from groq import Groq

#client = Groq(api_key='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')
example_sql_query = """
SELECT * FROM [dbo].[product_info];
"""

connection = pypyodbc.connect ("Driver={ODBC Driver 17 for SQL Server};"
                       "Server=localhost;"
                       "Database=business_data;"
                       "Trusted_Connection=yes;")
cursor = connection.cursor()

TABLE_NAME = "product_info"
SCHEMA_NAME = "[dbo]"

def extract_table_metadata():
    metadata_query= f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}'"
    cursor.execute(metadata_query)
    columns=cursor.fetchall()
    column_metadata={f"[{col[0]}]":col[1] for col in columns}
    print(f"Extracted metadata for table '{TABLE_NAME}':{column_metadata}")
    example_sql_query=f"SELECT{', '.join(column_metadata.keys())} FROM {TABLE_NAME} WHERE [CONDITION] LIMIT 10;"
    return column_metadata, example_sql_query

#extract only SQL query from the response
def extract_sql_from_response(response):
    queries=re.findall(r"SELECT.*?;", response, re.IGNORECASE | re.DOTALL)
    if queries:
        return queries[0].strip()
    else:
        raise ValueError("No valid SQL query found.")

#function to generate SQL query
def generate_sql_query(question, column_metadata, table_name, schema_name):
    full_table_name=f"{schema_name}.[{table_name}]"
    prompt=f"""Generate only 1 SQL query based on this question:{question}.
    The table to use is '{full_table_name}'. The columns are: {', '.join(column_metadata.keys())}.
    Use only the columns needed to answer the question, not all of them.
    Limit the data as much as possible and follows T-SQL syntax.
    Make sure to use the correct column names as provided with square brackets.
    Column [price] is money, so make sure you convert that to money before aggregating."""
    model_name='llama-3.1-8b-instant'
    chat_completion=client.chat.completions.create(
        messages=[
            {"role":"system", "content":"You are a helpful assistant."},
            {"role":"user", "content": prompt}
        ],
        model=model_name,
    )
    response=chat_completion.choices[0].message.content.strip()
    sql_query=extract_sql_from_response(response)
    return sql_query

#fetching data from SQL
def fetch_answer_from_db(sql_query):
    cursor.execute(sql_query)
    columns=[column[0] for column in cursor.description]
    result=cursor.fetchall()
    df=pd.DataFrame(result,columns=columns)
    return df

#agent where answer the question based on the DataFrame
def answer_question_from_df(question,df):
    df_json=df.to_json(orient='records')
    prompt=f"""Based on the following data, answer this question:{question}.
Here is the data:
{df_json}
Please provide a concise and accurate response."""
    model_name='llama-3.1-8b-instant'
    chat_completion=client.chat.completions.create(
        messages=[
            {"role":"system", "content":"You are a helpful assistant."},
            {"role":"user","content":prompt}
        ],
        model=model_name,
    )
    response=chat_completion.choices[0].message.content.strip()
    return response

def handle_question(question):
    column_metadata, example_sql_query=extract_table_metadata()
    sql_query = generate_sql_query(question, column_metadata, table_name, schema_name)
    print(f"Generated SQL Query: {sql_query}")
    df=fetch_answer_from_db(sql_query)
    print("Fetched Data in DataFrame:")
    print(df)
    answer=answer_question_from_df(question, df)
    print(f"Answer to the question: {answer}")

question = "What are the top 10 costly mobiles in our data and their details?"
handle_question(question)

connection.close()

Extracted metadata for table 'product_info':{'[product_id]': 'int', '[name]': 'varchar', '[features]': 'text', '[price]': 'decimal'}
Generated SQL Query: SELECT TOP 10 [name], [price]
FROM [dbo].[product_info]
ORDER BY CAST([price] AS MONEY) DESC;
Fetched Data in DataFrame:
                  name    price
0                I X96  1489.54
1           Work Max26  1468.41
2             In Pro81  1461.71
3      Financial Pro37  1457.13
4        Child Ultra59  1455.54
5  Environmental Pro34  1443.86
6        Account Pro30  1441.47
7            Money X56  1430.21
8       Assume Ultra49  1413.20
9            Whole X40  1391.03
Answer to the question: Based on the provided data, the top 10 costly mobiles and their details are:

1. I X96 - $1489.54
2. Work Max26 - $1468.41
3. In Pro81 - $1461.71
4. Financial Pro37 - $1457.13
5. Child Ultra59 - $1455.54
6. Environmental Pro34 - $1443.86
7. Account Pro30 - $1441.47
8. Money X56 - $1430.21
9. Assume Ultra49 - $1413.2
10. Whole X40 - $1391.03


In [128]:
#Building streamlit app

import streamlit as st
import pandas as pd
import pypyodbc
from groq import Groq
import re
from PIL import Image
import time

client=Groq(api_key='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')
st.set_page_config(layout="wide")

def connect_db():
    return pypyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                            "Server=localhost;"
                            "Database=business_data;"
                            "Trusted_Connection=yes;")

#main pic
image2= Image.open('AI Pic1.png')
st.sidebar.image(image2, use_column_width=True)

st.sidebar.header('Filters')
#tablename input
table_name=st.sidebar.text_input('Table Name', 'car_info_web')
#schema input
schema_name=st.sidebar.text_input('Schema','[dbo]')

#banner image
image=Image.open('AI Pic3.png')
st.image(image, use_column_width=True, caption="Sales amd AI Banner", output_format="PNG")

#setup 2 col
left_col, rigt_col = st.columns(2)

#to extract metadata from the table
def extract_table_metadata(table_name, schema_name):
    connection=connect_db()
    cursor=connection.cursor()
    metadata_query= f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{table_name}'"
    cursor.execute(metadata_query)
    columns=cursor.fetchall()
    column_metadata={f"[{col[0]}]":col[1] for col in columns}
    connection.close()
    return column_metadata

#function to generate SQL query
def generate_sql_query(question, column_metadata, table_name, schema_name):
    full_table_name=f"{schema_name}.[{table_name}]"
    prompt=f"""Generate only 1 SQL query based on this question:{question}.
    The table to use is '{full_table_name}'. The columns are: {', '.join(column_metadata.keys())}.
    Use only the columns needed to answer the question, not all of them.
    Column [price] is money, so make sure you convert that to money before aggregating."""
    model_name='llama-3.1-8b-instant'
    chat_completion=client.chat.completions.create(
        messages=[
            {"role":"system", "content":"You are a helpful assistant."},
            {"role":"user", "content": prompt}
        ],
        model=model_name,
    )
    response=chat_completion.choices[0].message.content.strip()
    return extract_sql_from_response(response)

#extract only SQL query from the response
def extract_sql_from_response(response):
    queries=re.findall(r"SELECT.*?;", response, re.IGNORECASE | re.DOTALL)
    if queries:
        return queries[0].strip()
    else:
        raise ValueError("No valid SQL query found.")

#fetching data from SQL
def fetch_answer_from_db(sql_query):
    connection=connect_db()
    cursor=connection.cursor()
    cursor.execute(sql_query)
    columns=[column[0] for column in cursor.description]
    result=cursor.fetchall()
    df=pd.DataFrame(result,columns=columns)
    connection.close()
    return df

#agent where answer the question based on the DataFrame
def answer_question_from_df(question,df):
    df_json=df.to_json(orient='records')
    prompt=f"Based on the following data, answer this question:{question}. Here is the data: {df_json}"
    model_name='llama-3.1-8b-instant'
    chat_completion=client.chat.completions.create(
        messages=[
            {"role":"system", "content":"You are a helpful assistant."},
            {"role":"user","content":prompt}
        ],
        model=model_name,
    )
    return chat_completion.choices[0].message.content.strip()

#retrying the function process if error occurs or the output is not legit
def run_with_retries(question, retries=3):
    success=False
    attempt=0
    result_df=None
    answer=None
    while attempt < retries and not success:
        try:
            attempt+=1
            column_metadata=extract_table_metadata(table_name, schema_name)
            sql_query=generate_sql_query(question, column_metadata, table_name, schema_name)
            #display the generated SQL query
            st.subheader(f"Generated SQL Query (Attempt {attempt})")
            st.code(sql_query)
            result_df=fetch_answer_from_db(sql_query)
            answer=answer_question_from_df(question, result_df)
            success=True #set as success if no error occurs
        except Exception as e:
            st.warning(f"Attempt {attempt} failed with error: {e}")
            time.sleep(1)
    if success:
        return result_df, answer
    else:
        st.error("Failed to process the request after three attempts.")
        return None, None

#question,input and answer set in left column
with left_col:
    st.header("Ask a Question")
    question=st.text_input("Enter your Question", "What are the costliest mobiles in our data with the price details?")
    if st.button('Submit'):
        df, answer = run_with_retries(question)
        if df is not None and answer is not None:
            st.subheader("Answer")
            st.write(answer)

#displaying the DataFrame on the right column
with right_col:
    st.header("Extracted Data")
    if 'df' in locals() and df is not None:
        st.dataframe(df)